In [8]:
from occ_examples.core_geometry_face_recognition_from_stepfile import *
from OCC.Core.GProp import GProp_GProps
import OCC.Core.BRepGProp as brp
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.STEPCAFControl import STEPCAFControl_Reader
from OCC.Core.TDF import TDF_LabelSequence
from OCC.Core.TDataStd import TDataStd_Name
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.XCAFApp import XCAFApp_Application
from OCC.Core.RWMesh import rwmesh

from pathlib import Path
from tqdm import tqdm


In [14]:
step_path = r"E:\gnn_data\step_files\100548_adapter_tos_naar_dappie_v1.step"
shp = read_step_file(step_path)
t2 = TopologyExplorer(shp)

In [19]:
len(list(t2.faces())), len(list(t2.edges())), len(list(t2.vertices()))

(87, 250, 165)

In [21]:
face_properties = {}
# Surface Area
for face_idx, face in enumerate(t2.faces()):
    face_properties[face_idx] = {}
    surface_props = GProp_GProps()
    brp.brepgprop.SurfaceProperties(face, surface_props)
    face_properties[face_idx]["area"] = surface_props.Mass()

    #Perimeter
    topo_explorer = TopologyExplorer(face)
    perimeter = 0.0
    for edge in topo_explorer.edges():
        edge_props = GProp_GProps()
        brp.brepgprop.LinearProperties(edge, edge_props)
        perimeter += edge_props.Mass()
    face_properties[face_idx]['perimeter'] = perimeter
    # Compactness (how close to a circle a face is)
    if face_properties[face_idx]['perimeter'] > 0 and face_properties[face_idx]['area'] > 0:
        face_properties[face_idx]["compactness"] = (4 * 3.14159 * face_properties[face_idx][
        'area']) / (face_properties[face_idx]['perimeter'] ** 2)

In [23]:
for edge in t2.edges():
    edge_props = GProp_GProps()
    brp.brepgprop.LinearProperties(edge, edge_props)
    print("Edge Length:", edge_props.Mass())

    edge_props = GProp_GProps()
    brp.brepgprop.LinearProperties(edge, edge_props)
    print("Edge Length:", edge_props.Length())

Edge Length: 15.891703228486831


AttributeError: 'GProp_GProps' object has no attribute 'Length'

In [10]:
broken_files = []
for file in tqdm(list(Path(step_path).parent.glob("*.*"))):
    try:
        shp = read_step_file(str(file))
        t = TopologyExplorer(shp)
        if not t.faces() or len(list(t.faces())) < 2:
            broken_files.append(file.name)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        broken_files.append(file.name)

100%|██████████| 63095/63095 [2:16:41<00:00,  7.69it/s]   


In [12]:
for file in broken_files:
    file_path = Path(step_path).parent / file
    new_file_path = Path("E:\gnn_data\step_broken") / file
    file_path.rename(new_file_path)

In [19]:
props = GProp_GProps()
brp.brepgprop.SurfaceProperties(faces[0], props)
print("Area:", props.Mass())

Area: 54.35581793659813


In [52]:
face0 = TopologyExplorer(faces[0])
face1 = TopologyExplorer(faces[1])
edges0 = list(face0.edges())
edges1 = list(face1.edges())
for edge in edges0:
    props = GProp_GProps()
    brp.brepgprop.LinearProperties(edge, props)
    print("Edge Length:", props.Mass())
for edge in edges1:
    props = GProp_GProps()
    brp.brepgprop.LinearProperties(edge, props)
    print("Edge Length:", props.Mass())

Edge Length: 18.849555921538762
Edge Length: 2.8836657034709
Edge Length: 18.849555921538762
Edge Length: 64.40264939859075
Edge Length: 44.84560681146358


In [53]:
for index, face in enumerate(faces):
    for edge in TopologyExplorer(face).edges():
        props = GProp_GProps()
        brp.brepgprop.LinearProperties(edge, props)
        print(f"Face {index} Edge Length:", props.Mass())

Face 0 Edge Length: 18.849555921538762
Face 0 Edge Length: 2.8836657034709
Face 0 Edge Length: 18.849555921538762
Face 1 Edge Length: 64.40264939859075
Face 1 Edge Length: 44.84560681146358
Face 2 Edge Length: 46.06203148693354
Face 2 Edge Length: 43.98229715025709
Face 3 Edge Length: 18.849555921538762
Face 3 Edge Length: 2.4414137103525335
Face 3 Edge Length: 15.99918890693674
Face 3 Edge Length: 15.99918890693674
Face 4 Edge Length: 31.99837781387348
Face 4 Edge Length: 16.2269
Face 4 Edge Length: 15.99918890693674
Face 4 Edge Length: 15.99918890693674
Face 5 Edge Length: 27.1282499043012
Face 5 Edge Length: 37.27439949428956
Face 5 Edge Length: 52.504444561972456
Face 6 Edge Length: 25.731322611488455
Face 6 Edge Length: 25.731322611488455
Face 6 Edge Length: 1.4943017369797889
Face 6 Edge Length: 23.10619859482299
Face 6 Edge Length: 23.10619859482299
Face 7 Edge Length: 45.39601384437251
Face 7 Edge Length: 1.7432173819622991
Face 7 Edge Length: 45.39601384437251
Face 8 Edge Leng